##Conexión SQLite3 base de datos

Esta función establece una conexión con una base de datos SQLite cuya ruta se pasa como argumento. Si la conexión es exitosa, devuelve tanto el objeto de conexión como el cursor. Si falla, imprime un mensaje de error y retorna None en ambos casos.

Argumentos:
  *   path (str): Ruta del archivo al que se quiere conectar.







In [1]:
def conectar_bd(path):
    """Establece una conexión con la base de datos SQLite."""
    try:
        con = sql.connect(path)
        cursor = con.cursor()
        print(f"Conexión exitosa a la base de datos (copia): {path}")
        return con, cursor
    except sql.Error as e:
        print(f"Error al conectar a la base de datos (copia) {path}: {e}")
        return None, None

# Listar tablas

Función que consulta y muestra los nombres de todas las tablas presentes en la base de datos.

Argumentos:
*   cursor: Objeto de conexión a la base de datos.

In [91]:
def listar_tablas(cursor):
    """Lista las tablas disponibles en la base de datos."""
    if cursor:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tablas = cursor.fetchall()
        print("\nTablas encontradas en la base de datos:")
        for tabla in tablas:
            print(f"* {tabla[0]}")
        return [t[0] for t in tablas]
    return []

# Explorar tabla

Funciones que nos permiten explora, de forma general, una tabla específica en la base de datos SQLite.

Argumentos:
*   cursor: Objeto de conexión a la base de datos.
*   nombre_tabla (str): El nombre de la tabla a explorar.
*   num_filas (int): Cantidad de filas a visualizar, por defecto 5.

In [92]:
# Ver el esquema de la tabla
def explorar_esquema_tabla(cursor, nombre_tabla):
    """Muestra el esquema de una tabla específica."""
    if cursor:
        print(f"\n ESQUEMA DE LA TABLA '{nombre_tabla}'")
        cursor.execute(f"PRAGMA table_info('{nombre_tabla}');")
        columnas = cursor.fetchall()
        for columna in columnas:
            print(f"\n  - Nombre Columna: {columna[1]}")
            print(f"    Tipo de Dato: {columna[2]}")
            print(f"    ¿Permite Valores Nulos? (0 es sí - 1 es no): {columna[3]}")

In [ ]:
# Contar el número total de filas
def contar_filas_tabla(cursor, nombre_tabla):
    """Cuenta el número total de filas en una tabla."""
    if cursor:
        cursor.execute(f"SELECT COUNT(*) FROM {nombre_tabla};")
        num_filas = cursor.fetchone()[0]
        print(f"\n NÚMERO TOTAL DE FILAS EN '{nombre_tabla}': {num_filas}")
        return num_filas
    return 0

# Mostrar las primeras 5 filas
def mostrar_primeras_filas_sql(cursor, nombre_tabla, num_filas=5):
    """Muestra las primeras N filas de una tabla usando SQL."""
    if cursor:
        cursor.execute(f"SELECT * FROM {nombre_tabla} LIMIT {num_filas};")
        primeras_filas = cursor.fetchall()
        if primeras_filas:
            print(f"\n PRIMERAS {num_filas} FILAS DE LA TABLA '{nombre_tabla}'")
            for fila in primeras_filas:
                print(fila)
        else:
            print(f"\nLA TABLA '{nombre_tabla}' ESTÁ VACÍA.")

# Moda

Calcula la moda de una lista de números o elementos.

Argumento:
* lista (list): Una lista de valores.

Retorna None si la lista está vacía. Si hay múltiples modas, retorna la primera encontrada por Counter.

In [93]:
def calcular_moda(lista):
    if not lista:
        return None
    contador = Counter(lista)
    moda = contador.most_common(1)[0][0]
    return moda

# Conteo de usuarios y peliculas

Calcula la cantidad de usuarios y peliculas que estan presentes en las bases de datos.

Argumento:
*   cursor: Objeto de conexión a la base de datos.

In [94]:
def conteo_usuarios_peliculas(cursor):
    """Cuenta el número total de usuarios y películas."""
    if cursor:
        cursor.execute("SELECT COUNT(DISTINCT userId), COUNT(DISTINCT movieId) FROM ratings;")
        num_usuarios, num_peliculas = cursor.fetchone()
        print(f"Número total de usuarios únicos: {num_usuarios}")
        print(f"Número total de películas únicas: {num_peliculas}")

# Distribución de calificaciones

Función que consulta y muestra la distribución de las calificaciones en la tabla 'ratings'.

Argumentos:
*   cursor: Objeto de conexión a la base de datos.



In [95]:
def distribucion_calificaciones(cursor):
    """Devuelve la distribución de calificaciones en un DataFrame."""
    if cursor:
        cursor.execute("SELECT rating, COUNT(*) AS frecuencia FROM ratings GROUP BY rating ORDER BY rating;")
        distribucion = cursor.fetchall()
        df_distribucion = pd.DataFrame(distribucion, columns=['rating', 'frecuencia'])
        return df_distribucion
    return pd.DataFrame(columns=['rating', 'frecuencia'])

# Top de películas más valoradas

Función que consulta y muestra las N películas con la mayor cantidad de valoraciones en la base de datos.

Argumentos:
*   cursor: Objeto de conexión a la base de datos.
*   n (int): El número de películas a mostrar, por defecto 10.

In [96]:
def top_n_peliculas_valoradas(cursor, n=10):
    """Devuelve las N películas con más valoraciones en un DataFrame."""
    if cursor:
        cursor.execute(f"""
            SELECT m.title, COUNT(r.movieId) AS total_valoraciones, m.movieId
            FROM ratings r
            JOIN movies m ON r.movieId = m.movieId
            GROUP BY m.movieId, m.title
            ORDER BY total_valoraciones DESC
            LIMIT {n};
        """)
        top_movies = cursor.fetchall()
        df_top_movies = pd.DataFrame(top_movies, columns=['title', 'total_valoraciones', 'movieId'])
        return df_top_movies
    return pd.DataFrame(columns=['title', 'total_valoraciones', 'movieId'])

# Separar géneros

Función que toma el DataFrame de películas con una columna de géneros separada por '|' y la expande en columnas booleanas para cada género único.

Argumentos:
*   df (pd.DataFrame): El DataFrame de películas con la columna 'genres'.

Retorna:
*   pd.DataFrame: Un nuevo DataFrame con las columnas de género añadidas (como enteros 0 o 1) y la columna 'genres' eliminada.



In [97]:
def separar_generos(df):
    """Separa la columna de géneros en columnas booleanas."""
    genres_series = df['genres'].str.split('|')
    te = TransactionEncoder()
    genres_array = te.fit(genres_series).transform(genres_series)
    df_genres = pd.DataFrame(genres_array, columns=te.columns_)
    df_processed = pd.concat([df, df_genres], axis=1).drop(columns='genres')
    df_processed = df_processed.astype({col: 'int' for col in df_genres.columns if df_genres[col].dtype == 'bool'})
    return df_processed

# Extraer año de estreno

Función que extrae el año del título de la película en el DataFrame y lo guarda en una nueva columna 'year'. También limpia la columna 'title' removiendo el año entre paréntesis.

Argumentos:
*  df (pd.DataFrame): El DataFrame de películas con la columna 'title'.

Retorna:
*   pd.DataFrame: El DataFrame con la columna 'year' añadida y la columna 'title' limpiada.

In [98]:
def extraer_anio(df):
    """Extrae el año del título de la película y lo limpia."""
    year_extracted = df['title'].str.extract(r'\((\d{4})\)')[0].astype('Int64')
    df['year'] = year_extracted
    df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()
    return df

# Reorganizar columnas de películas

Función que reorganiza las columnas del DataFrame de películas para tener 'movieId', 'title' y 'year' al inicio, seguidas por las columnas de género.

Argumentos:
*   df (pd.DataFrame): El DataFrame de películas.

Retorna:
*   pd.DataFrame: El DataFrame con las columnas reorganizadas.

In [99]:
def reorganizar_columnas_peliculas(df):
    """Reorganiza las columnas del DataFrame de películas."""
    generos_cols = [col for col in df.columns if col not in ['movieId', 'title', 'year']]
    df_reordered = df[['movieId', 'title', 'year'] + generos_cols]
    return df_reordered

# Preprocesar ratings

Función que preprocesa el DataFrame de ratings renombrando columnas y convirtiendo la columna de timestamp a un formato de fecha y hora.

Argumentos:
*   df (pd.DataFrame): El DataFrame de ratings con las columnas 'userId', 'movieId', 'rating' y 'timestamp'.



Retorna:
*   pd.DataFrame: El DataFrame de ratings preprocesado con las columnas renombradas y la columna 'fecha_vista' añadida.

In [100]:
def preprocesar_ratings(df):
    """Preprocesa el DataFrame de ratings."""
    df.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
    df['fecha_vista'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.drop(columns='timestamp')
    return df

# Eliminar películas sin año y genero

Funciónes que eliminan las filas donde la columna 'year' y 'genere' tiene un valor nulo (NaN).

Argumentos:
*   df (pd.DataFrame): El DataFrame de películas con la columna 'year'.
*   df (pd.DataFrame): El DataFrame de películas con la columna '(no genres listed)'.

Retorna:
*   pd.DataFrame: Un nuevo DataFrame sin las películas cuyo año es nulo.
*   pd.DataFrame: Un nuevo DataFrame sin las películas sin género listado.

In [101]:
def eliminar_peliculas_sin_anio(df):
    """Elimina las películas con año faltante."""
    df_filtered = df.dropna(subset=['year']).copy()
    print(f"\nNúmero de películas después de eliminar sin año: {len(df_filtered)}")
    return df_filtered

def eliminar_peliculas_sin_genero(df):
    """Elimina las películas sin género listado."""
    df_filtered = df[df['(no genres listed)'] == 0].copy()
    print(f"\nNúmero de películas después de eliminar sin género: {len(df_filtered)}")
    return df_filtered

#Películas por calificación

Función que consulta y muestra las películas que tienen una calificación específica dada por los usuarios.

Argumentos:
* con (sqlite3.Connection): La conexión a la base de datos SQLite.
* rating (float): El valor de la calificación para filtrar las películas (por defecto es 5).

In [102]:
def consultar_rating_con_titulo(con, rating=5):
    import pandas as pd
    import matplotlib.pyplot as plt

    # Consulta SQL para traer ratings del valor dado y títulos de películas
    query = f"""
        SELECT r.movieId, m.title, r.rating
        FROM ratings r
        JOIN movies m ON r.movieId = m.movieId
        WHERE r.rating = {rating}
    """
    df_rating = pd.read_sql(query, con)

    # Verificar si se encontraron resultados
    if df_rating.empty:
        print(f"No se encontraron películas con calificación {rating}.")
        return df_rating

In [103]:
def consultar_rating_con_titulo1(con, rating=5):
    import pandas as pd
    # Consulta SQL para traer ratings del valor dado y títulos de películas
    query = f"""
        SELECT r.movieId, m.title, r.rating
        FROM ratings r
        JOIN movies m ON r.movieId = m.movieId
        WHERE r.rating = {rating}
    """
    df_rating = pd.read_sql(query, con)
        # Verificar si se encontraron resultados
    if df_rating.empty:
        print(f"No se encontraron películas con calificación {rating}.")
    return df_rating